In [87]:
import pandas as pd
import numpy as np
import plotly.express as px
import fpcia as fp

# Getaround EDA : analyse exploratoire des données

Cette analyse de données doit permettre de répondre aux deux questions suivantes.
**Notre Product Manager doit encore trancher sur les points suivants :**
- **seuil** : quelle doit être la durée minimale du délai entre deux locations ?
- **périmètre** : faut-il activer cette fonctionnalité pour tous les véhicules ou uniquement pour les véhicules Connect ?
On peut également approfondir l'exploration en s'appuyant sur les questions :
- **impact** : quel est l'impact de cette fonctionnalité sur le nombre de locations ?
- Quelle part des revenus des propriétaires serait potentiellement affectée par cette fonctionnalité ?
- Combien de locations seraient impactées en fonction du seuil et du périmètre choisis ?
- À quelle fréquence les conducteurs sont-ils en retard pour le check-in suivant ? Quel est l’impact pour le conducteur suivant ?
- Combien de situations problématiques seraient résolues selon le seuil et le périmètre retenus ?



## Analyse du dataset sur les retards

| field name	|   Comment	|   
| ----------- | ----------- |
| rental_id	|       Unique identifier of the rental	|   
| car_id	    |       Unique identifier of the car	|   
| checkin_type |   	Flow used for both checkin and checkout. (ie. access and return the car) <br> mobile = rental agreement signed on the owner's smartphone <br> connect = car equiped with the Connect technology , opened by the driver with his smartphone.<br> **Note:** paper contracts were excluded from the data as we have no data on their delay at checkout and it's negligible use case |   
| state	   |        canceled means that the rental did not happen (was canceled by the driver or the owner).	|   
| delay_at_checkout_in_minutes	|    Difference in minutes between the rental end time requested by the driver when booking the car and the actual time the driver completed the checkout. Negative values |mean that the driver returned the car in advance.|   
| previous_ended_rental_id	|    id of the previous ended rental of the car (NULL when no previous rental or delay with previous rental higher than 12 hours)	|   
| time_delta_with_previous_rental_in_minutes	|    Difference in minutes between this rental planned start time and the previous rental planned end time (when lower than 12 hours, NULL if higher)	|   
        

In [ ]:
#df=pd.read_csv("../data/raw/get_around_delay_analysis.csv",sep=";")
delays_df = pd.read_excel("../data/raw/get_around_delay_analysis.xlsx")
print(fp.eda.summary(delays_df))
display(delays_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21310 entries, 0 to 21309
Data columns (total 7 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   rental_id                                   21310 non-null  int64  
 1   car_id                                      21310 non-null  int64  
 2   checkin_type                                21310 non-null  object 
 3   state                                       21310 non-null  object 
 4   delay_at_checkout_in_minutes                16346 non-null  float64
 5   previous_ended_rental_id                    1841 non-null   float64
 6   time_delta_with_previous_rental_in_minutes  1841 non-null   float64
dtypes: float64(3), int64(2), object(2)
memory usage: 1.1+ MB
________________________________________________
Data Start
   rental_id  car_id checkin_type     state  delay_at_checkout_in_minutes  \
0     505000  363965       mobile  c

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
0,505000,363965,mobile,canceled,NaN,NaN,NaN
1,507750,269550,mobile,ended,-81.0,NaN,NaN
2,508131,359049,connect,ended,70.0,NaN,NaN
3,508865,299063,connect,canceled,NaN,NaN,NaN
4,511440,313932,mobile,ended,NaN,NaN,NaN


In [89]:
# Créer la colonne "impact" :
# True si le retard de la location précédente > buffer prévu
delays_df['impact'] = np.where(
    delays_df['delay_at_checkout_in_minutes'] > delays_df['time_delta_with_previous_rental_in_minutes'],
    True,
    False
)

Concernant les valeurs nombreuses de temps non consigné, l'hypothèse faite est de les prendre comme des temps nul. Le dataset est donc transformé.

In [26]:
delays_df["delay_at_checkout_in_minutes"]=delays_df["delay_at_checkout_in_minutes"].fillna(0)
delays_df["time_delta_with_previous_rental_in_minutes"]=delays_df["time_delta_with_previous_rental_in_minutes"].fillna(0)

In [27]:
print(f" il y a {len(delays_df['rental_id'].unique())} location dans le dataset de 21310 lignes")
print(f" il y a {len(delays_df['car_id'].unique())} voitures dans le dataset de 21310 lignes")


 il y a 21310 location dans le dataset de 21310 lignes
 il y a 8143 voitures dans le dataset de 21310 lignes


In [98]:
# Comptage global par état
groupby = delays_df.groupby("checkin_type")
get_type = pd.DataFrame({"start_type": groupby['checkin_type'].count()})
plot_connect=get_type.reset_index()

# Pie chart
fig=px.pie(
    plot_connect,
    names='checkin_type',
    values='start_type',
    title="Répartition des types de checkin au global",           # Supprime le label par défaut
)
fig.show()

In [99]:
plot_connect

,checkin_type,start_type
0,connect,4307
1,mobile,17003


Dans le tableau, on remarque que dans les écarts sur l'heure de rendu ( _delay_at_checkout_in_minutes_ ), nous avons des temps de retard lorsque la valeur est positive et des temps de rendu en avance lorsque le temps est négatif.

In [68]:
cancel=delays_df[delays_df.state == "canceled"].count()
done=delays_df[delays_df.state == "ended"].count()
print("il y a {} locations annulées et {} locations terminées".format(cancel[0], done[0]))
early_delay_value_df = delays_df[delays_df.delay_at_checkout_in_minutes < 0]
print(f" il y a {len(early_delay_value_df)} locations avec un retour de véhicule en avance.") 
print(f"Cela représente {len(early_delay_value_df)/len(delays_df)*100:.2f}% du dataset")
late_delay_value_df = delays_df[delays_df.delay_at_checkout_in_minutes > 0]
print(f" il y a {len(late_delay_value_df)} locations avec un retour de véhicule en retard")
print(f" cela représente {len(late_delay_value_df)/len(delays_df)*100:.2f}% du dataset")


il y a 3265 locations annulées et 18045 locations terminées
 il y a 6820 locations avec un retour de véhicule en avance.
Cela représente 32.00% du dataset
 il y a 9404 locations avec un retour de véhicule en retard
 cela représente 44.13% du dataset


/var/folders/6m/4bfx39_j0519_3g3r0sp6fbw0000gn/T/ipykernel_13789/3226837653.py:3: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



 L'identifiant de la location précédente est très peu rempli pour être utile. Seulement   8,6% = (21310-19469)/21310 ont des valeurs remplies. La colonne _previous_ended_rental_id_ est donc supprimée de notre analyse ainsi que l'identifiant spécifique de la location qui n'a pas d'interêt.


In [29]:
delays_df.drop("previous_ended_rental_id", axis=1, inplace=True)

In [37]:
groupby = delays_df.groupby("car_id")
anomaly_cars = pd.DataFrame({"delay_at_checkout_in_minutes": groupby['delay_at_checkout_in_minutes'].sum(),
                    "occurence de retard": groupby['delay_at_checkout_in_minutes'].apply(lambda x: (x>0).sum()),
                    "occurence de retour en avance": groupby['delay_at_checkout_in_minutes'].apply(lambda x: (x<0).sum())})
anomaly_cars.reset_index(inplace=True)


In [38]:
anomaly_cars.describe()

,car_id,delay_at_checkout_in_minutes,occurence de retard,occurence de retour en avance
count,8143.000000,8143.000000,8143.000000,8143.000000
mean,350029.779074,119.842933,1.154857,0.837529
std,60426.145732,1444.054180,1.341921,1.332199
min,159250.000000,-22433.000000,0.000000,0.000000
25%,317118.500000,-26.000000,0.000000,0.000000
50%,369928.000000,1.000000,1.000000,0.000000
75%,397744.500000,102.000000,2.000000,1.000000
max,417675.000000,71084.000000,15.000000,16.000000


In [59]:
anomaly_late_rental = delays_df[delays_df["delay_at_checkout_in_minutes"]>0.0]
px.histogram(anomaly_late_rental, x="delay_at_checkout_in_minutes",nbins=10000)


In [58]:
anomaly_early_rental = delays_df[delays_df["delay_at_checkout_in_minutes"]<0.0]
px.histogram(anomaly_early_rental, x="delay_at_checkout_in_minutes",nbins=10000)

In [66]:
fig = px.scatter(anomaly_early_rental, x="delay_at_checkout_in_minutes", y="time_delta_with_previous_rental_in_minutes", color="state", symbol="checkin_type")
fig.show()

In [72]:
cancellations_and_deltas_dict = {"time_delta_with_previous_rental_in_minutes": [], "num_cancelled_rides_at_that_time_delta": []}

cancelled_has_previous_rental_deltas = (delays_df[delays_df.state == "canceled"])["time_delta_with_previous_rental_in_minutes"]

for d in range(int(max(delays_df.time_delta_with_previous_rental_in_minutes.to_list()))): # d is for delay
    cancellations_and_deltas_dict["time_delta_with_previous_rental_in_minutes"].append(d)
    num_cancellations = len(cancelled_has_previous_rental_deltas[cancelled_has_previous_rental_deltas <= d])
    cancellations_and_deltas_dict["num_cancelled_rides_at_that_time_delta"].append(num_cancellations)
cancellations_and_deltas_df = pd.DataFrame(cancellations_and_deltas_dict)
fig = px.line(
    data_frame=cancellations_and_deltas_df,
    x="time_delta_with_previous_rental_in_minutes",
    y="num_cancelled_rides_at_that_time_delta",
)
fig.show()

## Analyse du dataset sur les prix de location

La première colonne du dataset n'est pas nommé. Je la nomme donc _id_ car il s'agit d'un identifiant unique du véhicule loué mais ce n'est pas l'identifiant nommé _car_id_ du dataset précédent concernant les retards de retour de véhicule.

In [73]:
pricing_df = pd.read_csv("../data/raw/get_around_pricing_project.csv",sep=",")
print(fp.eda.summary(pricing_df))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4843 entries, 0 to 4842
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   id                         4843 non-null   int64 
 1   model_key                  4843 non-null   object
 2   mileage                    4843 non-null   int64 
 3   engine_power               4843 non-null   int64 
 4   fuel                       4843 non-null   object
 5   paint_color                4843 non-null   object
 6   car_type                   4843 non-null   object
 7   private_parking_available  4843 non-null   bool  
 8   has_gps                    4843 non-null   bool  
 9   has_air_conditioning       4843 non-null   bool  
 10  automatic_car              4843 non-null   bool  
 11  has_getaround_connect      4843 non-null   bool  
 12  has_speed_regulator        4843 non-null   bool  
 13  winter_tires               4843 non-null   bool  
 14  rental_p

In [100]:
# Statistiques basiques sur le dataset original

print("Number of rows : {}".format(pricing_df.shape[0]))
print("Number of columns : {}".format(pricing_df.shape[1]))

print("\nBasics statistics: ")
display(pricing_df.describe(include="all"))

print("\nPercentage of missing values: ")
display(100 * pricing_df.isnull().sum() / pricing_df.shape[0])

print("\nPercentage of duplicated lines: ")
print(pricing_df.duplicated().sum())

Number of rows : 4843
Number of columns : 15

Basics statistics: 


,id,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
count,4843.000000,4843,4.843000e+03,4843.00000,4843,4843,4843,4843,4843,4843,4843,4843,4843,4843,4843.000000
unique,NaN,28,NaN,NaN,4,10,8,2,2,2,2,2,2,2,NaN
top,NaN,Citroën,NaN,NaN,diesel,black,estate,True,True,False,False,False,False,True,NaN
freq,NaN,969,NaN,NaN,4641,1633,1606,2662,3839,3865,3881,2613,3674,4514,NaN
mean,2421.000000,NaN,1.409628e+05,128.98823,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121.214536
std,1398.198007,NaN,6.019674e+04,38.99336,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.568268
min,0.000000,NaN,-6.400000e+01,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.000000
25%,1210.500000,NaN,1.029135e+05,100.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.000000
50%,2421.000000,NaN,1.410800e+05,120.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,119.000000
75%,3631.500000,NaN,1.751955e+05,135.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,136.000000



Percentage of missing values: 


id                           0.0
model_key                    0.0
mileage                      0.0
engine_power                 0.0
fuel                         0.0
paint_color                  0.0
car_type                     0.0
private_parking_available    0.0
has_gps                      0.0
has_air_conditioning         0.0
automatic_car                0.0
has_getaround_connect        0.0
has_speed_regulator          0.0
winter_tires                 0.0
rental_price_per_day         0.0
dtype: float64


Percentage of duplicated lines: 
0


In [101]:
#IQR pour les distributions asymétriques

Q1 = pricing_df['rental_price_per_day'].quantile(0.25)
Q3 = pricing_df['rental_price_per_day'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"Outliers min : {lower_bound}, Outliers max : {upper_bound}")

# Histogramme
fig = px.histogram(pricing_df, x="rental_price_per_day")

# Ajouter les lignes
fig.add_vline(x=lower_bound, line_dash='dash', line_color='red')
fig.add_vline(x=upper_bound, line_dash='dash', line_color='red')

fig.show()

Outliers min : 56.0, Outliers max : 184.0


In [106]:
#IQR pour les distributions asymétriques
Q1 = pricing_df['mileage'].quantile(0.25)
Q3 = pricing_df['mileage'].quantile(0.75)
IQR = Q3 - Q1

# Bornes
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"Outliers min : {lower_bound}, Outliers max : {upper_bound}")

# Histogramme
fig = px.histogram(pricing_df, x="mileage")

# Lignes limites
fig.add_vline(x=lower_bound, line_dash='dash', line_color='red')
fig.add_vline(x=upper_bound, line_dash='dash', line_color='red')

fig.show()


Outliers min : -5509.5, Outliers max : 283618.5


In [107]:
display(pricing_df['model_key'].value_counts())

model_key
Citroën        969
Renault        916
BMW            827
Peugeot        642
Audi           526
Nissan         275
Mitsubishi     231
Mercedes        97
Volkswagen      65
Toyota          53
SEAT            46
Subaru          44
PGO             33
Ferrari         33
Opel            33
Maserati        18
Suzuki           8
Porsche          6
Ford             5
KIA Motors       3
Alfa Romeo       3
Fiat             2
Lexus            2
Lamborghini      2
Mazda            1
Honda            1
Mini             1
Yamaha           1
Name: count, dtype: int64

In [108]:
mean_price_per_brand = pricing_df.groupby("model_key")["rental_price_per_day"].mean().sort_values()

fig = px.bar(
    mean_price_per_brand,
    x=mean_price_per_brand.values,
    y=mean_price_per_brand.index,
    orientation='h',
    title="Prix moyen de location par marque",
    labels={
        "x": "Prix moyen",
        "y": "Marque"
    }
)

fig.update_layout(height=len(mean_price_per_brand) * 25)
fig.show()

In [109]:
display(pricing_df['fuel'].value_counts())
print(pricing_df['fuel'].unique())

fuel
diesel           4641
petrol            191
hybrid_petrol       8
electro             3
Name: count, dtype: int64

['diesel' 'petrol' 'hybrid_petrol' 'electro']


In [110]:
mean_price_per_brand = pricing_df.groupby("fuel")["rental_price_per_day"].mean().sort_values()

fig = px.bar(
    mean_price_per_brand,
    x=mean_price_per_brand.values,
    y=mean_price_per_brand.index,
    orientation='h',
    title="Prix moyen de location par energie",
    labels={
        "x": "Prix moyen",
        "y": "Energie"
    }
)

fig.update_layout(height=len(mean_price_per_brand) * 100)
fig.show()

### Conclusion
- Il n'est malheureusement pas possible de faire de lien entre ce dataframe et celui du delay : pas de correspondance avec la colonne 'car_id' ('Unnamed: 0' est une autre séquence)
- rental_price_per_day : aucune valeur ne semble aberrante, les loueurs peuvent mettre des locations très (trop) élevées ou faibles, mais le but de ce pricing optimization est justement de les aider à s'ajuster
- mileage : la valeur -64 est aberrante (impossible), la valeur 1000376 est extrème et semble peu probable (valeurs sont à supprimer du dataset)
- engine_power : la valeur nulle est aberrante (impossible), les deux valeurs à 25 pour des Porsche est assez étonnantes ce sont des hybrid_petrol, valeurs abérrantes!

___

In [8]:
fig = px.histogram(df, x="delay_at_checkout_in_minutes", nbins=100)
fig.update_traces(xbins=dict( # bins used for histogram
        start=-720.0,
        end=720.0,
        size=5
    ))
fig.show()
df_ended = df[df["checkin_type"]=="mobile"][["rental_id","delay_at_checkout_in_minutes"]]
fig = px.histogram(df_ended, x="delay_at_checkout_in_minutes", nbins=100)
fig.update_traces(xbins=dict( # bins used for histogram
        start=-720.0,
        end=720.0,
        size=5
    ))
fig.show()  

In [9]:
df_ended = df[df["state"]=="ended"][["rental_id","delay_at_checkout_in_minutes"]]
fig = px.histogram(df_ended, x="delay_at_checkout_in_minutes", nbins=100)
fig.update_traces(xbins=dict( # bins used for histogram
        start=-720.0,
        end=720.0,
        size=5
    ))
fig.show()  
df_ended = df[df["checkin_type"]=="connect"][["rental_id","delay_at_checkout_in_minutes"]]
fig = px.histogram(df_ended, x="delay_at_checkout_in_minutes", nbins=100)
fig.update_traces(xbins=dict( # bins used for histogram
        start=-720.0,
        end=720.0,
        size=5
    ))
fig.show()  
df_ended = df[df["state"]=="canceled"][["rental_id","time_delta_with_previous_rental_in_minutes"]]
fig = px.histogram(df_ended, x="time_delta_with_previous_rental_in_minutes", nbins=100)
fig.update_traces(xbins=dict( # bins used for histogram
        start=-720.0,
        end=720.0,
        size=5
    ))
fig.show()  

In [10]:
df[df["state"]=="canceled"]

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,Unnamed: 7,Unnamed: 8
0,505000,363965,mobile,canceled,NaN,NaN,NaN,NaN,NaN
3,508865,299063,connect,canceled,NaN,NaN,NaN,NaN,NaN
8,512475,322502,mobile,canceled,NaN,NaN,NaN,NaN,NaN
10,513743,330658,mobile,canceled,NaN,NaN,NaN,NaN,NaN
11,514161,366037,connect,canceled,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
21283,569325,345079,mobile,canceled,NaN,NaN,NaN,NaN,NaN
21287,569764,405347,mobile,canceled,NaN,NaN,NaN,NaN,NaN
21288,570001,386413,connect,canceled,NaN,NaN,NaN,NaN,NaN
21297,571481,311841,mobile,canceled,NaN,NaN,NaN,NaN,NaN


In [11]:
df_canceled = df[df["state"]=="canceled"][["rental_id","delay_at_checkout_in_minutes"]]
fig = px.histogram(df_canceled, x="delay_at_checkout_in_minutes", nbins=100)

fig.show()  
df_cancelled

NameError: name 'df_cancelled' is not defined

In [ ]:
df_canceled = df[(df["delay_at_checkout_in_minutes"] >= -60) & (df["delay_at_checkout_in_minutes"] <= 60)]  

In [ ]:
df_canceled 

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,Unnamed: 7,Unnamed: 8
6,511639,370585,connect,ended,-15.0,563782.0,570.0,NaN,NaN
7,512303,371242,mobile,ended,-44.0,NaN,NaN,NaN,NaN
9,513434,256528,connect,ended,23.0,NaN,NaN,NaN,NaN
13,515147,257466,mobile,ended,15.0,NaN,NaN,NaN,NaN
19,519491,312389,mobile,ended,58.0,545639.0,420.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
21292,570440,292303,mobile,ended,32.0,NaN,NaN,NaN,NaN
21293,570957,390479,mobile,ended,7.0,NaN,NaN,NaN,NaN
21294,571154,400293,mobile,ended,2.0,NaN,NaN,NaN,NaN
21296,571359,357612,mobile,ended,52.0,NaN,NaN,NaN,NaN


In [ ]:
df_analysis=df[(df["state"]=="canceled") & df["time_delta_with_previous_rental_in_minutes"]>0 ]

In [ ]:
df_analysis[["rental_id","car_id","checkin_type","state","time_delta_with_previous_rental_in_minutes"]]

,rental_id,car_id,checkin_type,state,time_delta_with_previous_rental_in_minutes
204,543768,374169,connect,canceled,210.0
242,546160,352528,connect,canceled,630.0
504,564627,341431,mobile,canceled,150.0
637,568657,317378,connect,canceled,210.0
669,516550,377700,mobile,canceled,720.0
...,...,...,...,...,...
21022,560787,413181,mobile,canceled,150.0
21172,566228,390871,connect,canceled,60.0
21230,569706,245154,connect,canceled,660.0
21269,568049,381499,connect,canceled,720.0


In [ ]:
fig = px.histogram(df_analysis, x="time_delta_with_previous_rental_in_minutes", nbins=200)
fig.update_traces(xbins=dict( # bins used for histogram
        start=0,
        end=1000.0,
        size=30
    ))
fig.show()